# Context specific (static) gene regulatory network (GRN) analysis

This notebooks performs analysis of the context specific GRNs we made with dictys. 

## Load GRNs

In [ ]:
# Removes CPU usage limit by some jupyter versions
import os
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Dictys
from dictys.net import network
from dictys.plot.static import compute_spec,fig_heatmap_top,fig_subnet
from dictys.plot import dotplot


os.environ['KMP_AFFINITY'] = ''

work_dir = '../../data/dictys/ExNeu/'
os.makedirs(work_dir, exist_ok=True)
d0=network.from_file(f'{work_dir}/results/static.h5')

## Regulation marker discovery based on target count & specificity between cell subsets
This analysis uses the reconstructed contest specific GRNs to find marker TFs based on their regulatory activity. These *regulation marker TFs* can provide information to annotate each cell subset and indicate the function of these regulation markers.

### Plot TF GRN activity by broad cluster

In [ ]:
#Regulation markers based on TF regulatory activity (by target count) and its specificity
#Color map of specificity
cmap='spring_r'
#Node sizes to draw legend in dot plot
node_size_deg=[20,100,200,500]
#Transformation from target count to node size
node_size_map_deg=lambda x:x*(x<=10)+(10+(x-10)*500/10/490)*(x>10)

#Find marker genes based on regulatory activity specificity
#Arguments control how many top markers to show and the cutoffs for target count, its specificity at the most abundant cell subset, and its entropy over all cell subsets
deg,deg_spec,cpm,cpm_spec,regmarker,regmarker_state=compute_spec(d0,nmax=15,vmin=10,ncut=0.3,min_entropy=0.7)
#Draw dot plot with regulatory activity and its specificity
fig,ax=dotplot(deg.loc[regmarker].iloc[::-1],deg_spec.loc[regmarker].iloc[::-1],size_transform=node_size_map_deg,sizes=np.array(node_size_deg),cmap=cmap)


## Heatmap of top target genes of select TFs in their corresponding cell types


In [ ]:
#TF-cell type pairs selected from the last step
tf=[('Fos','CA1'), ('Fos','CA2_3'), ('Fos','DG'),
    ('Junb','CA1'), ('Junb','CA2_3'), ('Junb','DG'),
   ('Nr4a1','CA1'), ('Nr4a1','CA2_3'), ('Nr4a1','DG'),
   ]
#Number of top targets for each TF
ntop=20
#Searching for top activated target genes only
direction=1

fig,fig_colorbar,net=fig_heatmap_top(d0,tf,ntop=ntop,direction=direction,aspect=1,gann='all')


## Draw target gene subnetwork from a TF
We can further zoom in to a particular TF in the relevant context and look at its targets.

### Example 1: Fos in CA1
As before, we first draw a large figure and annotate all genes to determine which we ultimately need.

In [ ]:
fig=plt.figure(figsize=(12,12))
ax=fig.add_subplot(111)
nodes,edges=fig_subnet(d0,ax,'CA1',regulators=['Fos'],annotate='all', sparsity=0.001)

The data matrices include node locations and edge strengths.

In [ ]:
nodes.head()

In [ ]:
edges.sort_values("Strength", ascending = False)

You can also draw the upstream regulators of FOS by constraining the target genes.

In [ ]:
fig=plt.figure()
ax=fig.add_subplot(111)
nodes,edges=fig_subnet(d0,ax,'CA1',targets=['Fos'],annotate='all')


### Example 2: Fos in DentateGyrus

In [ ]:
fig=plt.figure(figsize=(12,12))
ax=fig.add_subplot(111)
nodes,edges=fig_subnet(d0,ax,'DG',regulators=['Fos'],annotate='all', sparsity=0.01)


In [ ]:
edges.sort_values("Strength", ascending = False)

## Exporting networks to folder
You can export all context specific networks to tsv files for custom analyses.

In [ ]:
#Folder to export networks to
output_folder='../../results/dictys/ExNeu/'
#Choose which sparsities (proportion of edges present) to export network
#Use None to export continuous network
sparsities=[None,0.01, 0.005]
d0.export(output_folder,sparsities=sparsities)


The export folder is structured as below.

In [ ]:
!ls ../../results/dictys/ExNeu/

In [ ]:
!ls ../../results/dictys/ExNeu/sparsity=0.01